In [1]:
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.3.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:13 http://ppa.launchpad.net

In [2]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-12-23 10:45:22--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.68MB/s    in 0.5s    

2022-12-23 10:45:23 (1.68 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [4]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
books_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Books_v1_00.tsv.gz"), sep="\t", header=True)
books_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   25933450| RJOVP071AVAJO|0439873800|      84656342|There Was an Old ...|           Books|          5|            0|          0|   N|                Y|          Five Stars|I love it and so ...| 2015-08-31|
|         US|    1801372|R1ORGBETCDW3AI|1623953553|     729938122|      I Saw a Friend|           Books|          5|    

In [5]:
# Get the number of rows in the DataFrame.
books_df.count()

10319090

# Transform the Data

## Create the "review_id_table".

In [6]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
review_df = books_df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date")])
review_df.show()

+--------------+-----------+----------+--------------+--------------------+
|     review_id|customer_id|product_id|product_parent|to_date(review_date)|
+--------------+-----------+----------+--------------+--------------------+
| RJOVP071AVAJO|   25933450|0439873800|      84656342|          2015-08-31|
|R1ORGBETCDW3AI|    1801372|1623953553|     729938122|          2015-08-31|
| R7TNRFQAOUTX5|    5782091|142151981X|     678139048|          2015-08-31|
|R2GANXKDIFZ6OI|   32715830|014241543X|     712432151|          2015-08-31|
|R2NYB6C3R8LVN6|   14005703|1604600527|     800572372|          2015-08-31|
|R13U5PBJI1H94K|   36205738|0399170863|     559876774|          2015-08-31|
|R1H8UVH990F8VE|   44121167|1517007240|     299984591|          2015-08-31|
|R2MC0N30WZMRQ5|   16519255|0671728725|     821650353|          2015-08-31|
| R2NO2HXK16Y4J|   49361350|1111349533|     419457767|          2015-08-31|
|R245YIAVJK82ZL|   11423253|812211637X|     285887177|          2015-08-31|
| RFBMLBWQOZ

## Create the "products" Table

In [7]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df = books_df.select(["product_id", "product_title"]).drop_duplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0001622137|Tom Swift and His...|
|0001942611|Little Grey Rabbi...|
|0002553295|The Real Meaning ...|
|0002555166|Double Lives: Sta...|
|0004488172|Collins Scotland ...|
|0006382320|This is Orson Wel...|
|0006386563|Shadowlight: A Ph...|
|0007121024| The Hollow (Poirot)|
|0007145926|15-Minute Tai Chi...|
|0007148089|Blood and Roses: ...|
|0007155441|The Way of Egypti...|
|0007163657|Eating the Sun: H...|
|0007210604|Noddy and the New...|
|0007212860| Poirot Investigates|
|0007219075|  It's Me or the Dog|
|0007229151|The Times Origina...|
|000726741X|Haunted: Scariest...|
|0007274947|Fighter Heroes of...|
|0007285973|       Kashmir Shawl|
|0007301375|         The Reunion|
+----------+--------------------+
only showing top 20 rows



## Create the "customers" Table

In [8]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
customers_df = books_df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   37119503|             3|
|    3180632|             1|
|   14335758|             6|
|   44892935|             1|
|     740955|             1|
|   20228001|             4|
|   23810736|             1|
|   51291733|             1|
|   10632307|             1|
|   40758291|            21|
|   30142445|             2|
|   17356431|             8|
|   49948219|             3|
|   47840769|             2|
|   41826621|             2|
|   15073256|             1|
|   43801871|             5|
|    3223564|             1|
|   29191691|            22|
|   51052113|            79|
+-----------+--------------+
only showing top 20 rows



## Create the "vine_table".

In [9]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_df = books_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RJOVP071AVAJO|          5|            0|          0|   N|
|R1ORGBETCDW3AI|          5|            0|          0|   N|
| R7TNRFQAOUTX5|          5|            0|          0|   N|
|R2GANXKDIFZ6OI|          5|            0|          0|   N|
|R2NYB6C3R8LVN6|          5|            2|          2|   N|
|R13U5PBJI1H94K|          2|            1|          1|   N|
|R1H8UVH990F8VE|          5|            2|          2|   N|
|R2MC0N30WZMRQ5|          5|            0|          0|   N|
| R2NO2HXK16Y4J|          5|            0|          0|   N|
|R245YIAVJK82ZL|          5|            0|          0|   N|
| RFBMLBWQOZ1UM|          5|            0|          0|   N|
| RHE1FEWV3KTQH|          5|            0|          0|   N|
|R2BB9U7PHO7ETO|          5|            0|          1|   N|
|R28XJJP754GYXO|          4|            

# Load

In [10]:
mode = "append"
jdbc_url="jdbc:postgresql://192.168.1.42:5432/"
config = {"user":"postgres",
          "password": "postgres",
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vines', mode=mode, properties=config)